# Laboratorio Práctico: Automatización de Tareas de Base de Datos MySQL Usando Scripts de Shell

<span style="color: var(--vscode-foreground);">En este laboratorio, utilizarás la interfaz de línea de comandos (CLI) de MySQL para hacer copias de seguridad automáticamente de la base de datos y restaurar la base de datos cuando sea necesario.</span>  

## Software Utilizado en Este Laboratorio

Para completar este laboratorio, utilizarás el servicio de base de datos relacional MySQL disponible como parte de los Laboratorios de la Red de Habilidades de IBM (SN Labs). SN Labs es un entorno de laboratorio virtual utilizado en este curso.

## Base de Datos Utilizada en Este Laboratorio

Usarás una versión modificada de la base de datos Sakila para el laboratorio, que es una versión adaptada de la siguiente fuente: [https://dev.mysql.com/doc/sakila/en/](https:\dev.mysql.com\doc\sakila\en\?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDB0110ENSkillsNetwork24601058-2021-01-01) bajo la [licencia New BSD](https:\opensource.org\licenses\bsd-license.php?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDB0110ENSkillsNetwork24601058-2021-01-01) \[Copyright 2021 - Oracle Corporation\].

## Objetivos

Después de completar este laboratorio, podrás utilizar la línea de comandos de MySQL para:

- Crear el script de shell para respaldar la base de datos.
- Crear un trabajo cron para ejecutar el script de respaldo, creando así un archivo de respaldo.
- Truncar las tablas en la base de datos.
- Restaurar la base de datos utilizando el archivo de respaldo.

## Ejercicio

En este ejercicio crearás una base de datos, respaldarás la base de datos utilizando un script automatizado y finalmente la truncarás y restaurarás.

### area A: Crear una Base de Datos

1. Ve a **Terminal \> Nueva Terminal** para abrir un terminal desde el IDE en la nube lanzado lado a lado.
    
    ![new terminal](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0110EN-SkillsNetwork/labs/Lab%20-%20Getting%20started%20with%20MySQL%20command%20line/images/A.1.png)
    
2. Copia el comando a continuación y ejecútalo en el terminal para obtener el archivo [sakila\_mysql\_dump.sql](https:\cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud\IBM-DB0110EN-SkillsNetwork\datasets\sakila\sakila_mysql_dump.sql) en el IDE en la nube.
    

```bash
wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0110EN-SkillsNetwork/datasets/sakila/sakila_mysql_dump.sql
```

3. Inicia la sesión del servicio MySQL utilizando la directiva `Start MySQL in IDE button`.

 <span class="plugin-text" style="scrollbar-color: rgb(69, 74, 77) rgb(32, 35, 36); padding: 0px 0px 0px 2px; margin: 0px; box-sizing: border-box; display: inline;">Open MySQL Page in IDE</span>

4. En la página de lanzamiento, haz clic en el botón **Crear**.

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/nS9RnIfS89iFZs7oQ2onnQ/k2-1.png)

5. Abre el archivo `.my.cnf` como usuario root, con el editor nano en la terminal para configurar tu contraseña de mysql.

```
sudo nano ~/.my.cnf

```

6. Agrega la contraseña que anotaste en el paso anterior al archivo `.my.cnf`. Esto ayuda a no tener que ingresar la contraseña una y otra vez y mantiene la contraseña oculta en el archivo de configuración.

> Nota: Una vez que abras el archivo ~/.my.cnf, ingresa la línea `password = <Your MySQL Password>` y reemplaza la contraseña con la que anotaste anteriormente.

```bash
[client]host = mysqlport = 3306user = rootpassword = <Your MySQL Password>

```

7. Presiona Ctrl+O, luego seguido de la tecla Enter para guardar el archivo.
    
8. Presiona Ctrl+X para salir del editor nano.
    
9. Inicia la sesión del símbolo de comando mysql dentro de la sesión del servicio MySQL usando el comando a continuación en la terminal:
    

```bash
mysql

```

![image](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Automate%20Backup%20in%20MySQL/images/mysql.png)

Aquí puedes ver que puedes iniciar sesión en mysql sin ingresar la contraseña, ya que ya está configurada en el archivo ~/.my.cnf.

9. Crea una nueva base de datos **sakila** utilizando el comando a continuación en la terminal y procede a la Tarea B:

```sql
create database sakila;

```

![image](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Automate%20Backup%20in%20MySQL/images/sakila.png)

### Tarea B: Restaurar la Estructura y los Datos de una Tabla

1. Para usar la nueva base de datos sakila vacía, utiliza el siguiente comando en la terminal:
    
    ```sql
    use sakila;
    
    ```
    
    ![use sakila](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0110EN-SkillsNetwork/labs/Lab%20-%20Getting%20started%20with%20MySQL%20command%20line/images/B.1.png)
    
2. Restaura el archivo de volcado de mysql de sakila (que contiene las definiciones de tablas y datos de la base de datos sakila) a la nueva base de datos sakila vacía utilizando el siguiente comando en la terminal:
    
    ```sql
    source sakila_mysql_dump.sql;
    
    ```
    
3. Para verificar, enumera todos los nombres de las tablas de la base de datos sakila utilizando el siguiente comando en la terminal:
    
    ```sql
    SHOW FULL TABLES WHERE table_type = 'BASE TABLE';
    
    ```
    
    ![show full tables](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0110EN-SkillsNetwork/labs/Lab%20-%20Getting%20started%20with%20MySQL%20command%20line/images/C.1.png)
    

### Tarea C: Comprender el Proceso Involucrado en la Creación de Copias de Seguridad de Bases de Datos MySQL

Crearás un script de shell que haga lo siguiente:

- Escribir la base de datos en un archivo sql creado con una marca de tiempo, utilizando el comando `mysqldump`
    
- Comprimir el archivo sql en un archivo zip usando el comando `gzip`
    
- Eliminar el archivo sql utilizando el comando rm
    
- Eliminar la copia de seguridad después de 30 días
    

Antes de crear el script, entendamos cada uno de los bloques de comandos que agregarás al archivo.

- Para empezar, tienes una base de datos que deseas respaldar. Almacenarás el nombre de la base de datos en una variable.

```sql
DATABASE='sakila'
```

- Siempre es una buena práctica imprimir algunos registros, lo que puede ayudar en la depuración.

```sql
echo "Pulling Database: This may take a few minutes"
```

- También establecerás la carpeta de respaldo donde se almacenarán los archivos SQL y comprimidos.

```bash
backupfolder=/home/theia/backups

```

- Decidirás y establecerás el número de días que se necesitará mantener la copia de seguridad.

```bash
keep_day=30

```

- Establecerás el nombre del archivo SQL donde volcarás la base de datos como “all-database-“ seguido de la marca de tiempo actual y la extensión .sql, y el archivo zip en el que comprimirás el archivo SQL como “all-database-“ seguido de la marca de tiempo actual y la extensión .gz.

```
sqlfile=$backupfolder/all-database-$(date +%d-%m-%Y_%H-%M-%S).sql
zipfile=$backupfolder/all-database-$(date +%d-%m-%Y_%H-%M-%S).gz

```

- Ahora que se han creado todos los marcadores de posición, crearás la copia de seguridad SQL. En MySQL, esto se puede lograr con el comando `mysqldump`. Dependiendo de si la operación es exitosa o no, se imprime un registro. Si la operación es exitosa, comprimirás el archivo .sql en un .gz y eliminarás el archivo .sql.

```
if mysqldump  $DATABASE > $sqlfile ; then 
    echo 'Sql dump created'    
        # Compress backup     
        if gzip -c $sqlfile > $zipfile; then 
            echo 'The backup was successfully compressed'    
        else        
            echo 'Error compressing backupBackup was not created!'
            exit    
        fi
        rm $sqlfile 
else   
    echo 'pg_dump return non-zero code No backup was created!'    
    exit
fi

```

- Finalmente, eliminarás cualquier copia de seguridad que esté en el sistema por más tiempo del que decidiste retener la copia de seguridad.

```bash
find $backupfolder -mtime +$keep_day -delete

```

Ahora que has examinado los componentes y entendido lo que hace el script de shell, vamos a crear un archivo y guardar el script en él.

### Tarea D: Creando un Script de Shell para Copias de Seguridad de Base de Datos MySQL

1. Selecciona el menú **Archivo** y luego selecciona **Nuevo Archivo** para crear un nuevo script de shell llamado `sqlbackup.sh`.

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Automate%20Backup%20in%20MySQL/images/one.png)

Ingresa el siguiente código en el nuevo archivo:

```bash
#!/bin/sh
# The above line tells the interpreter this code needs to be run as a shell script.

# Set the database name to a variable. 
DATABASE='sakila'

# This will be printed on to the screen. In the case of cron job, it will be printed to the logs.
echo "Pulling Database: This may take a few minutes"

# Set the folder where the database backup will be stored
backupfolder=/home/theia/backups

# Number of days to store the backup 
keep_day=30

sqlfile=$backupfolder/all-database-$(date +%d-%m-%Y_%H-%M-%S).sql
zipfile=$backupfolder/all-database-$(date +%d-%m-%Y_%H-%M-%S).gz

# Create a backup
if mysqldump  $DATABASE > $sqlfile ; then   
    echo 'Sql dump created'    
    # Compress backup    
    if gzip -c $sqlfile > $zipfile; then
        echo 'The backup was successfully compressed'    
    else        
        echo 'Error compressing backupBackup was not created!'         
        exit    
    fi    
    rm $sqlfile 
else  
    echo 'pg_dump return non-zero code No backup was created!'    
    exit
fi

# Delete old backups 
find $backupfolder -mtime +$keep_day -delete

```

2. Guarda tu script usando la opción **Guardar** o presionando Comandos+S (en Mac) o Ctrl+S (Windows).
    
3. Ahora necesitas otorgar permiso de ejecución para el archivo del script de shell, al propietario (tú mismo), ejecutando el siguiente comando en la terminal. `u` significa usuario o creador, `x` significa ejecutar, y `r` significa permiso de lectura:
    

```bash
sudo chmod u+x+r sqlbackup.sh

```

![permisos ejecutables](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Automate%20Backup%20in%20MySQL/images/three.png)

4. Decidiste crear las copias de seguridad en una carpeta, pero esa carpeta no existe en el entorno. Necesitas crearla ejecutando el siguiente comando:

```bash
mkdir /home/theia/backups

```

### Tarea E: Configuración de un Cron Job

- Cron es un sistema que ayuda a los usuarios de Linux a programar cualquier tarea. Puede ser un script de shell o un simple comando bash.
    
- Un cron job nos ayuda a automatizar nuestras tareas rutinarias y puede ser por hora, diario, mensual, etc.
    
- Un crontab (tabla cron) es un archivo de texto que especifica el horario de los cron jobs.
    
- Cada línea en el archivo crontab contiene seis campos separados por un espacio seguidos del comando a ejecutar.
    

![crontab image](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Automate%20Backup%20in%20MySQL/images/crontabimage.png)


Los primeros cinco campos pueden contener uno o más valores separados por una coma o un rango de valores separados por un guion.

\* El operador asterisco significa cualquier valor o siempre. Si tienes el símbolo asterisco en el campo de Hora, significa que la tarea se realizará cada hora.

, El operador coma te permite especificar una lista de valores para repetición. Por ejemplo, si tienes 1,3,5 en el campo de Hora, la tarea se ejecutará a la 1 a.m., 3 a.m. y 5 a.m.

\- El operador guion te permite especificar un rango de valores. Si tienes 1-5 en el campo de Día de la semana, la tarea se ejecutará cada día laborable (de lunes a viernes).

/ El operador barra diagonal te permite especificar valores que se repetirán en un cierto intervalo entre ellos. Por ejemplo, si tienes \*/4 en el campo de Hora, significa que la acción se realizará cada cuatro horas. Es lo mismo que especificar 0,4,8,12,16,20. En lugar de un asterisco antes del operador de barra diagonal, también puedes usar un rango de valores. Por ejemplo, 1-30/10 significa lo mismo que 1,11,21.

Para entender cómo funciona un crontab, configuraremos un cron job que ocurre cada 2 minutos.

1. Para iniciar el crontab, ejecuta el siguiente comando en la terminal:

```bash
crontab -e

```

Esto abrirá un editor de crontab como se muestra a continuación.

![open cron editor](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Automate%20Backup%20in%20MySQL/images/seven.png)

2. Desplázate hasta la parte inferior de la página del editor utilizando la tecla de flecha hacia abajo y copia y pega el siguiente código:

```bash
*/2 * * * * /home/project/sqlbackup.sh > /home/project/backup.log

```

3. Presiona Ctrl+O seguido de la tecla Enter para guardar el archivo.
    
4. Presiona Ctrl+X para salir del editor de cron.
    

![salir del editor de cron](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Automate%20Backup%20in%20MySQL/images/eight.png)

5. El servicio cron debe ser iniciado explícitamente. Inicia el servicio cron ejecutando el siguiente comando:

```bash
sudo service cron start

```

6. Después de 2 minutos, ejecuta el siguiente comando para verificar si se han creado los archivos de respaldo:

```bash
ls -l /home/theia/backups

```

![backups](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Automate%20Backup%20in%20MySQL/images/nine.png)

> En un escenario de la vida real, el servicio cron es un servicio de larga duración que se ejecuta en segundo plano. Para detener el trabajo de cron, puedes ejecutar `sudo service cron stop`.

7. En este ejemplo, el cron está configurado para hacer una copia de seguridad cada 2 minutos. Detén el servicio cron utilizando el siguiente comando:

```bash
sudo service cron stop

```

### Ejercicio Práctico

1. Cambia el horario del crontab para crear una copia de seguridad cada semana el lunes a las 12:00 a.m.

<details><summary>Solución (Haz clic aquí) </summary>

0 0 * * 1 /home/project/sqlbackup.sh > /home/project/backup.log

</details>

2. Cambia el horario del crontab para crear una copia de seguridad cada día a las 6:00 a.m.

<details><summary>Solución (Haz clic aquí) </summary>

0 6 * * * /home/project/sqlbackup.sh > /home/project/backup.log

</details>

### Tarea F: Truncar las Tablas en la Base de Datos

Ahora que has automatizado la tarea de copia de seguridad, vamos a replicar un escenario donde los datos están corruptos o perdidos y eliminarás todos los datos en la base de datos y restaurarás los datos desde la copia de seguridad.

Crearemos un script de truncado que hace lo siguiente:

- Se conecta a mysql RDBMS utilizando las credenciales.
    
- Lista las tablas usando `show tables` y alimenta la salida usando el operador pipe (|) al siguiente comando.
    
- Itera a través de cada tabla usando un bucle while y trunca la tabla.
    

1. Crea un nuevo archivo llamado **truncate.sh** en home/project.

![new truncate file](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Automate%20Backup%20in%20MySQL/images/ten.png)

2. Copia este script y pégalo en el nuevo archivo.

```bash
#!/bin/sh 

DATABASE=sakila

mysql -Nse 'show tables' sakila | \   
    while read table; do mysql \    
    -e "use sakila;SET FOREIGN_KEY_CHECKS=0;truncate table $table;SET FOREIGN_KEY_CHECKS=1;" ;done

```

3. Cambia los permisos del archivo ejecutando el siguiente comando:

```bash
sudo chmod u+x+r truncate.sh

```

4. Ejecuta el script para truncar las tablas.

```bash
bash truncate.sh

```

5. Para verificar si las tablas en la base de datos están truncadas, inicie sesión en la base de datos con las credenciales.

```bash
mysql

```

La interfaz de mysql aparece

![mysql interface](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Automate%20Backup%20in%20MySQL/images/eleven.png)

6. Cambie a la base de datos **sakila**.

```sql
use sakila;
```

![sakila database](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Automate%20Backup%20in%20MySQL/images/twelve.png)

7. Verifica todas las tablas en la base de datos.

```sql
show tables;
```

![mostrar tablas](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Automate%20Backup%20in%20MySQL/images/twelvea.png)

8. Recupera todas las filas de la tabla de personal. Si el `truncate` fue exitoso, la salida debería ser un **Conjunto vacío**.

```sql
select * from staff;Copied!Wrap Toggled!
```

![recuperar todas las filas de la tabla de personal](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Automate%20Backup%20in%20MySQL/images/thirteen.png)

9. Sal del aviso de mysql.

```
\q
```

![salir de mysql](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0110EN-SkillsNetwork/labs/Lab%20-%20Getting%20started%20with%20MySQL%20command%20line/images/C.4.png)

### Tarea G: Restaurar la Base de Datos

Para restaurar la base de datos:

- Recoge un archivo zip comprimido presente en la carpeta de respaldo y descomprímelo para extraer el archivo sql utilizando el comando gunzip.
    
- Conéctate a la base de datos mysql y restaura la base de datos con el archivo sql.
    

1. En la ventana del terminal, ejecuta el siguiente comando para encontrar la lista de archivos de respaldo que se han creado.

```bash
ls -l /home/theia/backups
```

2. Seleccione el archivo del cual desea restaurar los datos y copie el nombre del archivo.
    
3. Descomprima el archivo y extraiga el archivo SQL del archivo de respaldo.
    

```bash
gunzip /home/theia/backups/<backup zip file name>
```

4. Pueble y restaure la base de datos con el archivo sql que resulta de la operación de descompresión.

```bash
mysql sakila < /home/theia/backups/<backup sql file name>Copied!Wrap Toggled!
```

Una vez que el comando se ejecute, se te pedirá que ingreses tu contraseña de inicio de sesión de mysql. Pega la contraseña que copiaste anteriormente usando Ctrl+V y presiona Enter.

5. Para verificar la base de datos restaurada, ve al aviso de mysql.

```bash
mysql
```

6. Usa la base de datos **sakila**:

```bash
use sakila;Copied!Wrap Toggled!
```

7. Selecciona todas las filas de cualquiera de las tablas, como se indica a continuación. Deberías encontrar que la base de datos está restaurada.

```sql
select * from staff;
```

![seleccionar del personal](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/MySQL/Lab%20-%20Automate%20Backup%20in%20MySQL/images/sixteen.png)

8. Sal de la sesión del símbolo del sistema de MySQL utilizando el comando a continuación en la terminal y procede a la Tarea D:

```sql
\q
```

### Ejercicio Práctico

1. Crea un script de shell que tome el nombre de la base de datos y el directorio de respaldo como parámetros y respalde la base de datos como `&lt;dbname&gt;_timestamp.sql` en el directorio de respaldo. Si la base de datos no existe, debe mostrar un mensaje apropiado. Si el directorio de respaldo no existe, debe crear uno.

<details><summary>Solución (Haz clic aquí) </summary>

```bash

dbname=$(mysql -e "SELECT SCHEMA_NAME FROM INFORMATION_SCHEMA.SCHEMATA WHERE SCHEMA_NAME = '$1'" | grep $1)

if [ ! -d $2 ]; then
    mkdir $2
fi

if [ $1 == $dbname ]; then
    sqlfile=$2/$1-$(date +%d-%m-%Y).sql    
    if mysqldump  $1 > $sqlfile ; then    
    echo 'Sql dump created'    
    else        
        echo 'Error creating backup!'    
    fi
else    
    echo "Database doesn't exist"
fi
```
</details>

2. Escribe un script de shell que tome el nombre de la base de datos y el archivo de script como parámetros y restaure la base de datos desde el archivo sql.

<details><summary>Solución (Haz clic aquí) </summary>

```bash

if [ -f $2 ]; then     
    dbname=$(mysql -e "SELECT SCHEMA_NAME FROM INFORMATION_SCHEMA.SCHEMATA WHERE SCHEMA_NAME = '$1'" | grep $1)    
    if [ $1 != $dbname ]; then        
        echo "Created DB as it didn't exist"        
        mysql -e "Create database $1"    
    fi    
    mysql -e "use $1"    
    mysql $1 < $2
else    
    echo "File doesn't exist"
fi
```
</details>

### Ejercicio Opcional

1. Puedes limpiar la carpeta de copias de seguridad utilizando el siguiente comando:

```bash
sudo rm -rfv /home/theia/backups
```